# Min Slektsanalyse / My Family Analysis

**🔒 PRIVAT**: Denne notebook inneholder analyser av sensitive familie-data / This notebook contains analyses of sensitive family data

---

## 🇳🇴 Norsk

### Instruksjoner

1. **Installer slektstre-pakken** først:
   ```bash
   pip install git+https://github.com/arvidl/slektstre.git
   ```

2. **Plasser dine datafiler** i `../data/`-mappen

3. **Aldri commit** denne notebook med reelle data til offentlige repos!

### Sikkerhet

- ✅ Alle datafiler er git-ignored
- ✅ Kun analyser (ikke data) committes
- ✅ Backup regelmessig (kryptert)
- ❌ Del aldri raw data offentlig

---

## 🇬🇧 English

### Instructions

1. **Install the slektstre package** first:
   ```bash
   pip install git+https://github.com/arvidl/slektstre.git
   ```

2. **Place your data files** in the `../data/` directory

3. **Never commit** this notebook with real data to public repos!

### Security

- ✅ All data files are git-ignored
- ✅ Only analyses (not data) are committed
- ✅ Backup regularly (encrypted)
- ❌ Never share raw data publicly


In [ ]:
# =============================================================================
# IMPORTER PAKKER / IMPORT PACKAGES
# =============================================================================

# Sjekk om vi kjører i Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔧 Kjører i Google Colab - installerer avhengigheter...")
    print("🔧 Running in Google Colab - installing dependencies...")
    
    # Installer nødvendige pakker
    %pip install -q networkx matplotlib plotly pydantic pyyaml pandas ipywidgets pillow kaleido
    
    # Klon repository
    !git clone https://github.com/arvidl/slektstre.git
    import sys
    sys.path.append('/content/slektstre/src')
    
except ImportError:
    IN_COLAB = False
    print("💻 Kjører lokalt / Running locally")
    import sys
    sys.path.append('../src')

# Importer slektstre-pakken
from slektstre import Slektstre
from slektstre.models import Person, Relationship

# Standard biblioteker
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

print("✅ Alle pakker importert / All packages imported")
print(f"📍 Miljø: {'Google Colab' if IN_COLAB else 'Lokal'}")
print(f"📍 Environment: {'Google Colab' if IN_COLAB else 'Local'}")


## 📊 Last Inn Familie-data / Load Family Data

**⚠️ VIKTIG**: Erstatt `familie.yaml` med din egen datafil / **IMPORTANT**: Replace `familie.yaml` with your own data file


In [ ]:
# =============================================================================
# LAST INN DATA / LOAD DATA
# =============================================================================

# Endre denne stien til din datafil / Change this path to your data file
DATA_FILE = "../data/familie.yaml"  # eller familie.json, familie.gedcom

try:
    # Last inn slektstreet
    slektstre = Slektstre()
    slektstre.load_from_file(DATA_FILE)
    
    print(f"✅ Data lastet inn fra: {DATA_FILE}")
    print(f"✅ Data loaded from: {DATA_FILE}")
    print(f"👥 Antall personer: {len(slektstre.persons)}")
    print(f"👥 Number of persons: {len(slektstre.persons)}")
    print(f"🔗 Antall relasjoner: {len(slektstre.relationships)}")
    print(f"🔗 Number of relationships: {len(slektstre.relationships)}")
    
except FileNotFoundError:
    print(f"❌ Fil ikke funnet: {DATA_FILE}")
    print(f"❌ File not found: {DATA_FILE}")
    print("💡 Plasser din familie-datafil i data/-mappen")
    print("💡 Place your family data file in the data/ directory")
    
except Exception as e:
    print(f"❌ Feil ved lasting av data: {e}")
    print(f"❌ Error loading data: {e}")
    print("💡 Sjekk at filformatet er korrekt (YAML, JSON, eller GEDCOM)")
    print("💡 Check that the file format is correct (YAML, JSON, or GEDCOM)")


## 📈 Grunnleggende Statistikk / Basic Statistics


In [ ]:
# =============================================================================
# GRUNNLEGGENDE STATISTIKK / BASIC STATISTICS
# =============================================================================

if 'slektstre' in locals() and slektstre.persons:
    
    # Konverter til DataFrame for analyse
    persons_data = []
    for person in slektstre.persons.values():
        persons_data.append({
            'id': person.id,
            'fornavn': person.first_name,
            'etternavn': person.last_name,
            'fødselsår': person.birth_date.year if person.birth_date else None,
            'kjønn': person.gender,
            'generasjon': slektstre.get_generation(person.id)
        })
    
    df_persons = pd.DataFrame(persons_data)
    
    print("📊 GRUNNLEGGENDE STATISTIKK / BASIC STATISTICS")
    print("=" * 50)
    
    # Antall personer per generasjon
    if 'generasjon' in df_persons.columns:
        gen_stats = df_persons['generasjon'].value_counts().sort_index()
        print(f"\n👥 Personer per generasjon / Persons per generation:")
        for gen, count in gen_stats.items():
            print(f"   Generasjon {gen}: {count} personer")
    
    # Kjønnsfordeling
    if 'kjønn' in df_persons.columns:
        gender_stats = df_persons['kjønn'].value_counts()
        print(f"\n⚥ Kjønnsfordeling / Gender distribution:")
        for gender, count in gender_stats.items():
            print(f"   {gender}: {count} personer")
    
    # Fødselsår-statistikk
    if 'fødselsår' in df_persons.columns:
        birth_years = df_persons['fødselsår'].dropna()
        if len(birth_years) > 0:
            print(f"\n📅 Fødselsår-statistikk / Birth year statistics:")
            print(f"   Tidligste: {int(birth_years.min())}")
            print(f"   Seneste: {int(birth_years.max())}")
            print(f"   Gjennomsnitt: {int(birth_years.mean())}")
    
    # Relasjonsstatistikk
    rel_types = {}
    for rel in slektstre.relationships:
        rel_type = rel.relationship_type
        rel_types[rel_type] = rel_types.get(rel_type, 0) + 1
    
    print(f"\n🔗 Relasjonstyper / Relationship types:")
    for rel_type, count in rel_types.items():
        print(f"   {rel_type}: {count}")
        
else:
    print("❌ Ingen data lastet inn / No data loaded")


## 🎨 Visualisering / Visualization

**💡 Tips**: Disse visualiseringene kan tilpasses etter dine behov / **💡 Tips**: These visualizations can be customized to your needs


In [ ]:
# =============================================================================
# VISUALISERING / VISUALIZATION
# =============================================================================

if 'slektstre' in locals() and slektstre.persons:
    
    # 1. Slektstre-visualisering
    print("🎨 Lager slektstre-visualisering...")
    print("🎨 Creating family tree visualization...")
    
    try:
        # Bruk slektstre-pakken sin visualisering
        fig = slektstre.visualize_tree()
        fig.show()
        
    except Exception as e:
        print(f"⚠️ Kunne ikke lage slektstre-visualisering: {e}")
        print(f"⚠️ Could not create family tree visualization: {e}")
    
    # 2. Generasjonsfordeling
    if 'df_persons' in locals() and 'generasjon' in df_persons.columns:
        plt.figure(figsize=(10, 6))
        gen_counts = df_persons['generasjon'].value_counts().sort_index()
        
        plt.subplot(1, 2, 1)
        gen_counts.plot(kind='bar', color='skyblue')
        plt.title('Personer per generasjon / Persons per generation')
        plt.xlabel('Generasjon / Generation')
        plt.ylabel('Antall / Count')
        plt.xticks(rotation=45)
        
        # 3. Kjønnsfordeling
        if 'kjønn' in df_persons.columns:
            plt.subplot(1, 2, 2)
            gender_counts = df_persons['kjønn'].value_counts()
            plt.pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%')
            plt.title('Kjønnsfordeling / Gender distribution')
        
        plt.tight_layout()
        plt.show()
    
    # 4. Fødselsår-tidslinje
    if 'df_persons' in locals() and 'fødselsår' in df_persons.columns:
        birth_years = df_persons['fødselsår'].dropna()
        if len(birth_years) > 0:
            plt.figure(figsize=(12, 6))
            plt.hist(birth_years, bins=20, color='lightgreen', alpha=0.7, edgecolor='black')
            plt.title('Fødselsår-fordeling / Birth year distribution')
            plt.xlabel('År / Year')
            plt.ylabel('Antall personer / Number of persons')
            plt.grid(True, alpha=0.3)
            plt.show()
    
    print("✅ Visualiseringer fullført / Visualizations completed")
    
else:
    print("❌ Ingen data tilgjengelig for visualisering / No data available for visualization")


## 🔍 Avanserte Analyser / Advanced Analyses

**💡 Eksempel**: Her kan du legge til dine egne analyser / **💡 Example**: Here you can add your own analyses


In [ ]:
# =============================================================================
# AVANSERTE ANALYSER / ADVANCED ANALYSES
# =============================================================================

if 'slektstre' in locals() and slektstre.persons:
    
    print("🔍 AVANSERTE ANALYSER / ADVANCED ANALYSES")
    print("=" * 50)
    
    # 1. Finn sentrale personer (høyest grad i nettverket)
    if slektstre.graph:
        degrees = dict(slektstre.graph.degree())
        if degrees:
            max_degree = max(degrees.values())
            central_persons = [pid for pid, deg in degrees.items() if deg == max_degree]
            
            print(f"\n🌟 Mest sentrale personer (grad {max_degree}):")
            print(f"🌟 Most central persons (degree {max_degree}):")
            for pid in central_persons:
                person = slektstre.persons.get(pid)
                if person:
                    print(f"   - {person.first_name} {person.last_name}")
    
    # 2. Finn lengste slektslinje
    try:
        longest_lineage = slektstre.find_longest_lineage()
        if longest_lineage:
            print(f"\n📏 Lengste slektslinje ({len(longest_lineage)} generasjoner):")
            print(f"📏 Longest lineage ({len(longest_lineage)} generations):")
            for i, pid in enumerate(longest_lineage):
                person = slektstre.persons.get(pid)
                if person:
                    print(f"   {i+1}. {person.first_name} {person.last_name}")
    except Exception as e:
        print(f"⚠️ Kunne ikke finne lengste slektslinje: {e}")
    
    # 3. Analyse av familiegrener
    try:
        branches = slektstre.identify_family_branches()
        print(f"\n🌳 Identifiserte familiegrener: {len(branches)}")
        print(f"🌳 Identified family branches: {len(branches)}")
        
        for i, branch in enumerate(branches[:5]):  # Vis kun første 5
            print(f"   Gren {i+1}: {len(branch)} personer")
    except Exception as e:
        print(f"⚠️ Kunne ikke identifisere familiegrener: {e}")
    
    # 4. Eksport til CSV for videre analyse
    if 'df_persons' in locals():
        export_file = "../data/analyse_eksport.csv"
        df_persons.to_csv(export_file, index=False)
        print(f"\n💾 Data eksportert til: {export_file}")
        print(f"💾 Data exported to: {export_file}")
        print("💡 Du kan åpne denne filen i Excel eller andre verktøy")
        print("💡 You can open this file in Excel or other tools")
    
    print("\n✅ Avanserte analyser fullført / Advanced analyses completed")
    
else:
    print("❌ Ingen data tilgjengelig for avanserte analyser / No data available for advanced analyses")


## 📝 Notater og Konklusjoner / Notes and Conclusions

**💡 Din analyse**: Legg til dine egne notater og konklusjoner her / **💡 Your analysis**: Add your own notes and conclusions here


### 🔒 Sikkerhetsnotater / Security Notes

- ✅ **Data sikkerhet**: Alle sensitive datafiler er git-ignored
- ✅ **Data security**: All sensitive data files are git-ignored
- ✅ **Backup**: Husk å ta regelmessige backups av dine data
- ✅ **Backup**: Remember to take regular backups of your data
- ❌ **Del aldri**: Raw data eller personlige detaljer offentlig
- ❌ **Never share**: Raw data or personal details publicly

### 📚 Neste Steg / Next Steps

1. **Utvid analysen**: Legg til flere visualiseringer og statistikk
2. **Expand analysis**: Add more visualizations and statistics
3. **Importer mer data**: Legg til flere personer og relasjoner
4. **Import more data**: Add more persons and relationships
5. **Del anonymiserte innsikter**: Med familie eller genealogi-samfunnet
6. **Share anonymized insights**: With family or genealogy community

### 🛠️ Tekniske Tips / Technical Tips

- **Performance**: For store slektstre, vurder å bruke sampling
- **Performance**: For large family trees, consider using sampling
- **Visualisering**: Tilpass farger og layout etter dine preferanser
- **Visualization**: Customize colors and layout to your preferences
- **Eksport**: Bruk CSV-eksport for videre analyse i andre verktøy
- **Export**: Use CSV export for further analysis in other tools
